## Open Exploration

Explanation to what we did + table of contents

### Import Libraries

In [1]:
# General libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy

# Import problem formulation
from problem_formulation import get_model_for_problem_formulation

# EMA workbench imports
from ema_workbench import (Model, Policy, MultiprocessingEvaluator, ScalarOutcome, RealParameter, 
                           IntegerParameter, CategoricalParameter, optimize, Scenario, 
                           Constant, SequentialEvaluator)
#from ema_workbench.em_framework.evaluators import perform_experiments,LHS, SOBOL, MORRIS, SequentialEvaluator, BaseEvaluator
from ema_workbench.em_framework.samplers import sample_uncertainties, sample_levers
from ema_workbench.util import ema_logging, save_results, load_results

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

## Worst Case Scenario 

In [2]:
# define the problem formulation between 0 and 9
# Problem formulation 6 is specific to Dike Ring 3
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [3]:
# Set uncertainties and levers variables
uncertainties = copy.deepcopy(dike_model.uncertainties)
levers = copy.deepcopy(dike_model.levers)

In [4]:
# function to fill empty policy 
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

# Create a reference policy where no action is taken
null_policy = [Policy('null_policy', **dict(get_do_nothing_dict()))]

In [5]:
n_scenarios = 100

ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=null_policy)

In [ ]:
#Save the results
save_results([experiments, outcomes], 'results/10000Scenarios_NullPolicy_PF6.tar.gz')
# load the worst case results for further analysis 
experiments, outcomes = load_results('results/10000Scenarios_NullPolicy_PF6.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_NullPolicy_PF6.tar.gz
[MainProcess/INFO] results loaded successfully from /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_NullPolicy_PF6.tar.gz


## Here actual Open exploration shit


### show that random policy is better than no policy

In [ ]:
# get the value ranges for policies 
for policy in dike_model.levers:
    print(repr(policy))

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [ ]:
# Create a policy where action is taken randomly
# compare the performance of that policy to the previously computed null_policy
import random # for randomly deciding to switch policy on/ off

random_policies = [Policy('alpha_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,4)})]

'''                Policy('beta_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,4)}),
                Policy('gamma_random_policy', **{'3_RfR 0':random.randint(0,1),
                                  '3_RfR 1':random.randint(0,1),
                                  '3_RfR 2':random.randint(0,1),
                                  'A.1_DikeIncrease 0':random.randint(0,10),
                                  'A.2_DikeIncrease 0':random.randint(0,10),
                                  'A.3_DikeIncrease 0':random.randint(0,10),
                                  'A.4_DikeIncrease 0':random.randint(0,10),
                                  'A.5_DikeIncrease 0':random.randint(0,10),
                                  'A.1_DikeIncrease 1':random.randint(0,10),
                                  'A.2_DikeIncrease 1':random.randint(0,10),
                                  'A.3_DikeIncrease 1':random.randint(0,10),
                                  'A.4_DikeIncrease 1':random.randint(0,10),
                                  'A.5_DikeIncrease 1':random.randint(0,10),
                                  'A.1_DikeIncrease 2':random.randint(0,10),
                                  'A.2_DikeIncrease 2':random.randint(0,10),
                                  'A.3_DikeIncrease 2':random.randint(0,10),
                                  'A.4_DikeIncrease 2':random.randint(0,10),
                                  'A.5_DikeIncrease 2':random.randint(0,10),
                                  'EWS_DaysToThreat':random.randint(0,4)})                  
                                  ]'''

"                Policy('beta_random_policy', **{'3_RfR 0':random.randint(0,1),\n                                  '3_RfR 1':random.randint(0,1),\n                                  '3_RfR 2':random.randint(0,1),\n                                  'A.1_DikeIncrease 0':random.randint(0,10),\n                                  'A.2_DikeIncrease 0':random.randint(0,10),\n                                  'A.3_DikeIncrease 0':random.randint(0,10),\n                                  'A.4_DikeIncrease 0':random.randint(0,10),\n                                  'A.5_DikeIncrease 0':random.randint(0,10),\n                                  'A.1_DikeIncrease 1':random.randint(0,10),\n                                  'A.2_DikeIncrease 1':random.randint(0,10),\n                                  'A.3_DikeIncrease 1':random.randint(0,10),\n                                  'A.4_DikeIncrease 1':random.randint(0,10),\n                                  'A.5_DikeIncrease 1':random.randint(0,10),\n       

In [ ]:
n_scenarios = 100

ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    random_experiments, random_outcomes = evaluator.perform_experiments(scenarios=n_scenarios, policies=1)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 1 policies * 1 model(s) = 100 experiments
100%|████████████████████████████████████████| 100/100 [00:14<00:00,  6.84it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [ ]:
#Save the results
save_results([random_experiments, random_outcomes], 'results/10000Scenarios_RandomPolicy_PF6.tar.gz')
# load the worst case results for further analysis 
random_experiments, random_outcomes = load_results('results/10000Scenarios_RandomPolicy_PF6.tar.gz')

[MainProcess/INFO] results saved successfully to /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_RandomPolicy_PF6.tar.gz
[MainProcess/INFO] results loaded successfully from /Users/philipmuller/Documents/GitHub/model-based-decision-making/final assignment/results/10000Scenarios_RandomPolicy_PF6.tar.gz


In [ ]:
random_outcomes = pd.DataFrame(random_outcomes).drop(columns=['A.1_Dike Investment Costs',
                                                              'A.2_Dike Investment Costs',
                                                              'A.3_Dike Investment Costs',
                                                              'A.4_Dike Investment Costs',
                                                              'A.5_Dike Investment Costs',
                                                              'RfR Total Costs',
                                                              'Expected Evacuation Costs'])

# put outcomes from null_policy into similar format
outcomes = pd.DataFrame(outcomes).drop(columns=['A.1_Dike Investment Costs',
                                                              'A.2_Dike Investment Costs',
                                                              'A.3_Dike Investment Costs',
                                                              'A.4_Dike Investment Costs',
                                                              'A.5_Dike Investment Costs',
                                                              'RfR Total Costs',
                                                              'Expected Evacuation Costs'])

In [ ]:
# statistical tests and heatmap of p_values
from scipy.stats import normaltest, mannwhitneyu, ttest_ind

# test for normal distribution. If test fails, use non-parametric test of central tendency 
for col in outcomes.columns: 
    statistic, p_value = normaltest(outcomes[col])
    if p_value >= .05: 
        print(f'normaltest failed for {col} with p={p_value}')
    
for col in random_outcomes.columns:
    statistic, p_value = normaltest(random_outcomes[col])
    if p_value >= 0.05:    
        print(f'normaltest failed for {col} with p={p_value}')

In [ ]:
p = pd.DataFrame(columns=outcomes.columns, index=['p_value'])
for col in p.columns: 
    statistic, p_value = ttest_ind(outcomes[col], random_outcomes[col])
    p[col] = p_value
p

,A.1_Expected Annual Damage,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Expected Number of Deaths
p_value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Here we could do a first visual analysis if we want

## PRIM

In [ ]:
random_outcomes

,A.1_Expected Annual Damage,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Expected Number of Deaths
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0


In [ ]:
from ema_workbench.analysis import prim

# Set the output to the category that we are interested in for this run
outcome = random_outcomes['A.3_Expected Number of Deaths']

x = random_experiments
y_limit = np.quantile(outcome, 0.9) # Setting output bound to be 90th percentile 
y = np.array([v > y_limit for v in outcome]) # Only select outputs that are above this threshold

prim_alg = prim.Prim(x, y, threshold=0.8) # run PRIM on bound=0.8
box1 = prim_alg.find_box() # Get the box that prim found for us

AssertionError: 

In [ ]:
# Show the peeling graph for this prim analysis
box1.show_tradeoff()
plt.show()

NameError: name 'box1' is not defined

In [ ]:
# Show scatter plots for this prim analysis
box1.show_pairs_scatter()
plt.show()

NameError: name 'box1' is not defined

In [ ]:
# Do dimensional stacking of results
from ema_workbench.analysis import dimensional_stacking

dimensional_stacking.create_pivot_plot(x, y, 2, nbins=5)
plt.show()

NameError: name 'x' is not defined